# Data cleaning Immobiliare.it

Importing the librares

In [631]:
import numpy as np
import pandas as pd
import re

I check the raw dataframe and the datatypes

In [632]:
df_raw = pd.read_csv('house_prices_italy.csv')
df_raw.head()

,Unnamed: 0,region,city,area,rooms,toilets,price
0,0,abruzzo,Pescara,89m²,3,1,€ 75.000
1,1,abruzzo,Spoltore,199m²,5+,3+,€ 235.000
2,2,abruzzo,Pescara,227m²,5,3+,€ 299.000
3,3,abruzzo,Appartamenti di nuova costruzione a Tortoreto,43m²,2 - 4,1,da € 165.000
4,4,abruzzo,Roseto degli Abruzzi,978m²,5+,3+,€ 1.500.000


In [633]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  40000 non-null  int64 
 1   region      40000 non-null  object
 2   city        40000 non-null  object
 3   area        39975 non-null  object
 4   rooms       39050 non-null  object
 5   toilets     40000 non-null  object
 6   price       40000 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.1+ MB


I know that I will need to change datatypes at least on 'area' and 'price'.

Are there any null values? let's see:

In [634]:
df_raw.isnull().sum()

Unnamed: 0      0
region          0
city            0
area           25
rooms         950
toilets         0
price           0
dtype: int64

I decided to drop the first column since it does not carry any info:

In [635]:
df=df_raw.drop("Unnamed: 0", axis=1)

In [636]:
df.head()

,region,city,area,rooms,toilets,price
0,abruzzo,Pescara,89m²,3,1,€ 75.000
1,abruzzo,Spoltore,199m²,5+,3+,€ 235.000
2,abruzzo,Pescara,227m²,5,3+,€ 299.000
3,abruzzo,Appartamenti di nuova costruzione a Tortoreto,43m²,2 - 4,1,da € 165.000
4,abruzzo,Roseto degli Abruzzi,978m²,5+,3+,€ 1.500.000


I force the "area" column as 'string', if not, later it will give errors.

In [637]:
df['area'] = df['area'].astype(str)

I do not like the column names. I rename them:

In [638]:
df.rename(columns={'area':'area[m2]', 'price':'price[€]'}, inplace=True)
df.sample(10)

,region,city,area[m2],rooms,toilets,price[€]
38738,veneto,Malo,118m²,5+,2,€ 140.000
2140,basilicata,Pisticci,138m²,5,1,"da € 77.475,00"
37017,valle-d-aosta,La Thuile,67m²,2,1,€ 330.000
6152,calabria,Cosenza,70m²,3,1,€ 79.000
19148,marche,Camerano,96m²,3,1,€ 129.000
12778,lazio,Roma,217m²,5,3,€ 990.000
27097,sardegna,Arzachena,200m²,5+,3+,Prezzo su richiesta
10729,friuli-venezia-giulia,Nimis,339m²,5,2,€ 280.000
11089,friuli-venezia-giulia,Forni Avoltri,71m²,3,1,€ 88.000
7086,calabria,Corigliano-Rossano,350m²,5,3,€ 239.000


Regex on "area" column. Let's start cleaning from here:

In [639]:
#compilo il pattern
p = re.compile('[0-9]+')


df['area[m2]'] = df['area[m2]'].apply(lambda x: 0 if p.search(x) is None else p.search(x).group())
df.head()

,region,city,area[m2],rooms,toilets,price[€]
0,abruzzo,Pescara,89,3,1,€ 75.000
1,abruzzo,Spoltore,199,5+,3+,€ 235.000
2,abruzzo,Pescara,227,5,3+,€ 299.000
3,abruzzo,Appartamenti di nuova costruzione a Tortoreto,43,2 - 4,1,da € 165.000
4,abruzzo,Roseto degli Abruzzi,978,5+,3+,€ 1.500.000


I convert 'area' column into 'Int64' dtype:

In [640]:
df['area[m2]'] = df['area[m2]'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   region    40000 non-null  object
 1   city      40000 non-null  object
 2   area[m2]  40000 non-null  int64 
 3   rooms     39050 non-null  object
 4   toilets   40000 non-null  object
 5   price[€]  40000 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.8+ MB


It's the turn for the "price" column:

In [641]:
df['price[€]'] = df['price[€]'].str.replace('.','', regex=False)
df['price[€]'] = df['price[€]'].str.replace('€','', regex= False)
df['price[€]']


0                      75000
1                     235000
2                     299000
3                 da  165000
4                    1500000
                ...         
39995                 890000
39996                  79000
39997                 260000
39998                 590000
39999    Prezzo su richiesta
Name: price[€], Length: 40000, dtype: object

In [642]:
p_price = re.compile('\d+')
df['price[€]'] = df['price[€]'].apply(lambda x: 0 if x.strip().isnumeric() == False else p.search(x).group())

In [643]:
df['price[€]'] = df['price[€]'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   region    40000 non-null  object
 1   city      40000 non-null  object
 2   area[m2]  40000 non-null  int64 
 3   rooms     39050 non-null  object
 4   toilets   40000 non-null  object
 5   price[€]  40000 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.8+ MB


In [644]:
df['area[m2]'] = df['area[m2]'].apply(lambda x: np.nan if x==0 else x)
df['price[€]'] = df['price[€]'].apply(lambda x: np.nan if x==0 else x)
df

,region,city,area[m2],rooms,toilets,price[€]
0,abruzzo,Pescara,89.0,3,1,75000.0
1,abruzzo,Spoltore,199.0,5+,3+,235000.0
2,abruzzo,Pescara,227.0,5,3+,299000.0
3,abruzzo,Appartamenti di nuova costruzione a Tortoreto,43.0,2 - 4,1,NaN
4,abruzzo,Roseto degli Abruzzi,978.0,5+,3+,1500000.0
...,...,...,...,...,...,...
39995,veneto,Caorle,106.0,5,2,890000.0
39996,veneto,Vicenza,48.0,2,1,79000.0
39997,veneto,Venezia,100.0,4,2,260000.0
39998,veneto,Bussolengo,305.0,5,3,590000.0


The 'rooms' and 'toilets' columns are still treated as strings. As they are in this moment one could use them as categorical values. I leave it this way.

I now rename the regions into a more appropriate manner:

In [645]:
regions_dict = {'abruzzo':'Abruzzo', 'basilicata':'Basilicata', 'campania':'Campania', 'calabria':'Calabria', 'emilia-romagna':'Emilia Romagna',
       'friuli-venezia-giulia':'Friuli-Venezia Giulia', 'lazio': 'Lazio', 'liguria':'Liguria', 'lombardia':'Lombardia', 'marche':'Marche',
       'molise':'Molise', 'piemonte':'Piemonte', 'puglia':'Puglia', 'sardegna':'Sardegna', 'sicilia':'Sicilia', 'toscana':'Toscana',
       'trentino-alto-adige': 'Trentino-Alto Adige', 'umbria':'Umbria', 'valle-d-aosta':'Valle d\'Aosta', 'veneto':'Veneto'}

df.replace({'region':regions_dict}, inplace=True)
df.isna().sum()


region         0
city           0
area[m2]      25
rooms        950
toilets        0
price[€]    4683
dtype: int64

In [646]:
df.sample(20)

,region,city,area[m2],rooms,toilets,price[€]
1986,Abruzzo,Roseto degli Abruzzi,168.0,5,2,278000.0
10243,Friuli-Venezia Giulia,Premariacco,470.0,5,3,270000.0
22538,Piemonte,Torino,75.0,3,1,130000.0
11080,Friuli-Venezia Giulia,Lestizza,240.0,5+,2,169000.0
36983,Valle d'Aosta,Ayas,59.0,2,1,240000.0
5197,Campania,Massa Lubrense,210.0,5,2,450000.0
6661,Calabria,Squillace,90.0,4,1,73000.0
34087,Umbria,Ville unifamiliari di nuova costruzione a Perugia,150.0,NaN,1,NaN
25377,Puglia,Bari,53.0,3,1,170000.0
32397,Trentino-Alto Adige,Merano,118.0,3,2,373000.0


I believe this is enough. It will be possible to load the data into a DataViz software and keep going from there with some visualizations

In [647]:
df.to_csv('house_prices_italy_cleaned.csv')